In [ ]:
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import matplotlib
matplotlib.rcParams['pdf.fonttype'] = 42 # for pdfs
import matplotlib.pyplot as plt
from pathlib import Path
import seaborn as sns
import pickle
from scipy import stats

import flexiznam as flz
from cottage_analysis.analysis import spheres
from cottage_analysis.pipelines import pipeline_utils
from v1_depth_analysis.v1_manuscript_2023 import depth_selectivity, closed_loop_rsof, get_session_list, depth_decoder
from v1_depth_analysis.v1_manuscript_2023 import common_utils as v1_common_utils

In [ ]:
VERSION = 6
SAVE_ROOT = Path(
    f"/camp/lab/znamenskiyp/home/shared/presentations/v1_manuscript_2023/ver{VERSION}"
)
SAVE_ROOT.mkdir(parents=True, exist_ok=True)
(SAVE_ROOT/"openloop_separate_recordings").mkdir(parents=True, exist_ok=True)

In [ ]:
import os
import warnings
from pandas.errors import PerformanceWarning
warnings.simplefilter(action='ignore', category=PerformanceWarning)
def extract_amplitude_separate_recordings(neurons_df):
    closedloop_prefix = "rsof_popt_closedloop_g2d_recording_closedloop_"
    openloop_prefix = "rsof_popt_openloop_actual_g2d_recording_openloop_"
    recording_number = np.nanmax([int(i[-1]) for i in neurons_df.columns[neurons_df.columns.str.contains("recording_")]]) + 1
    for i in range(recording_number): 
        neurons_df[f"amplitude_closedloop_recording{i}"] = np.nan
        neurons_df[f"amplitude_openloop_recording{i}"] = np.nan
    neurons_df["recording_order"] = [[np.nan]*recording_number]*len(neurons_df)
    recording_order = np.zeros((recording_number, len(neurons_df)))
    for col in neurons_df.columns:
        if openloop_prefix in col:
            openloop_num = int(col[-1])
            neurons_df[f"amplitude_openloop_recording{openloop_num}"] = neurons_df[col].apply(lambda x: np.exp(x[0]) + x[-1])
        if closedloop_prefix in col:
            closedloop_num = int(col[-1])
            neurons_df[f"amplitude_closedloop_recording{closedloop_num}"] = neurons_df[col].apply(lambda x: np.exp(x[0]) + x[-1])
            recording_order[closedloop_num,:] = 1
    neurons_df["recordings_order"] = recording_order.T.tolist()
    return neurons_df


def concatenate_neurons_df_for_separate_recordings(flexilims_session, session_list, cols, filename="neurons_df.pickle", read_iscell=True, verbose=True, recording_number_max=6):
    isess = 0
    for session in session_list:
        cols_new=cols
        neurons_ds = pipeline_utils.create_neurons_ds(
            session_name=session,
            flexilims_session=flexilims_session,
            project=None,
            conflicts="skip",
        )
        if os.path.exists(neurons_ds.path_full.parent / filename):
            neurons_df = pd.read_pickle(neurons_ds.path_full.parent / filename)
            neurons_df = extract_amplitude_separate_recordings(neurons_df)
            if cols is not None:
                cols_new = cols + neurons_df.columns[neurons_df.columns.str.contains("amplitude")].tolist()
                cols_new += ["recordings_order"]   
            if (cols is None) or (set(cols_new).issubset(neurons_df.columns.tolist())):
                if cols is None:
                    neurons_df = neurons_df
                else:
                    if len(neurons_df["recordings_order"].iloc[0]) > 2:
                        neurons_df = neurons_df[cols_new]
                        suite2p_ds = flz.get_datasets(
                            flexilims_session=flexilims_session,
                            origin_name=session,
                            dataset_type="suite2p_rois",
                            filter_datasets={"anatomical_only": 3},
                            allow_multiple=False,
                            return_dataseries=False,
                        )
                        if read_iscell:
                            iscell = np.load(
                                suite2p_ds.path_full / "plane0" / "iscell.npy",
                                allow_pickle=True,
                            )[:, 0]
                            neurons_df["iscell"] = iscell

                        neurons_df["session"] = session
                        
                        if recording_number_max > len(neurons_df["recordings_order"].iloc[0]):
                            for i in range(len(neurons_df["recordings_order"].iloc[0]), recording_number_max): 
                                neurons_df[f"amplitude_closedloop_recording{i}"] = np.nan
                                neurons_df[f"amplitude_openloop_recording{i}"] = np.nan
                        if isess == 0:
                            neurons_df_all = neurons_df
                        else:
                            neurons_df_all = pd.concat(
                                [neurons_df_all, neurons_df], 
                                ignore_index=True,
                            )

                        if verbose:
                            print(f"Finished concat {filename} from session {session}")
                        isess += 1
                    else:
                        print(f"ERROR: SESSION {session}: not more than 2 recordings")
            else:
                print(f"ERROR: SESSION {session}: specified cols not all in neurons_df")
        else:
            print(f"ERROR: SESSION {session}: {filename} not found")
            
    return neurons_df_all

In [ ]:


project = "hey2_3d-vision_foodres_20220101"
flexilims_session = flz.get_flexilims_session(project)     
neurons_df_all = concatenate_neurons_df_for_separate_recordings(flexilims_session=flexilims_session,
    session_list=[
    "PZAH8.2h_S20230224",
    "PZAH8.2h_S20230302",
    "PZAH8.2h_S20230303",
    "PZAH8.2h_S20230314",
    "PZAH8.2h_S20230321",
    
    "PZAH8.2i_S20230203",
    "PZAH8.2i_S20230209", 
    "PZAH8.2i_S20230216",
    "PZAH8.2i_S20230220", 
    "PZAH8.2i_S20230324",
    "PZAH8.2i_S20230330",
    "PZAH8.2i_S20230404",
    
    "PZAH8.2f_S20230206",
    "PZAH8.2f_S20230214",
    "PZAH8.2f_S20230223",
    "PZAH8.2f_S20230313",
    
    "PZAH10.2d_S20230602", 
    "PZAH10.2d_S20230608", 
    "PZAH10.2d_S20230613", 
    "PZAH10.2d_S20230623", 
    "PZAH10.2d_S20230818", 
    "PZAH10.2d_S20230821", 
    "PZAH10.2d_S20230920",
    "PZAH10.2d_S20230922", 
    
    "PZAH10.2f_S20230606", 
    "PZAH10.2f_S20230609", 
    "PZAH10.2f_S20230615", 
    "PZAH10.2f_S20230623",
    "PZAH10.2f_S20230627",
    "PZAH10.2f_S20230822", 
    "PZAH10.2f_S20230908", 
    "PZAH10.2f_S20230924",
    ], 
                                               cols=[
                                                    "roi",
                                                    "best_depth",
                                                    "preferred_depth_closedloop",
                                                    "preferred_depth_closedloop_crossval",
                                                    "depth_tuning_test_rsq_closedloop",
                                                    "depth_tuning_test_spearmanr_pval_closedloop",
                                                    "depth_tuning_test_spearmanr_rval_closedloop",
                                                    "preferred_RS_closedloop_g2d",
                                                    "preferred_RS_closedloop_crossval_g2d",
                                                    "preferred_OF_closedloop_g2d",
                                                    "preferred_OF_closedloop_crossval_g2d",
                                                    "rsof_rsq_closedloop_crossval_g2d",
                                                    "preferred_RS_openloop_actual_g2d",
                                                    "preferred_OF_openloop_actual_g2d",
                                                    "rsof_popt_closedloop_g2d",
                                                    "rsof_popt_openloop_actual_g2d",
                                                    "rsof_rsq_closedloop_g2d",
                                                    "rsof_rsq_openloop_actual_g2d",
                                                ], 
                                               filename="neurons_df.pickle", 
                                               read_iscell=True, 
                                               verbose=True)
# neurons_df_all.to_pickle(SAVE_ROOT / "openloop_separate_recordings" / "neurons_df_multiple_recordings_only.pickle")

In [ ]:
neurons_df_all = pd.read_pickle(SAVE_ROOT / "openloop_separate_recordings" / "neurons_df_multiple_recordings_only.pickle")

In [ ]:
import warnings
warnings.simplefilter(action='ignore')
def find_alternating_sequence(recordings_order):
    recordings_order = np.array(recordings_order)   #[0,1,1,0,1,0,0,1]
    zero_pos = np.where(recordings_order ==0)[0]
    one_pos = np.where(recordings_order ==1)[0]
    recordings_order_trim = recordings_order[one_pos[0]:] # discard zeros before the first one
    # find where ones and zeros are changing
    seq = pd.Series(recordings_order_trim)[pd.Series(recordings_order_trim).diff() != 0].index + one_pos[0]
    return seq

# plot amplitude for each recording
select_neurons = (
    (neurons_df_all["iscell"] == 1)
    & (neurons_df_all["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1)
    & (neurons_df_all["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
    # & ((neurons_df_all["rsof_rsq_closedloop_g2d"] > 0.02) | (neurons_df_all["rsof_rsq_openloop_actual_g2d"] > 0.02))
)

for i in range(5):
    for recording_type in ["closedloop", "openloop"]:
        globals()[f"amplitudes_{recording_type}_{i}"] = []
for session in neurons_df_all.session.unique():
    seq = find_alternating_sequence(neurons_df_all[neurons_df_all.session == session].recordings_order.iloc[0])
    print(neurons_df_all[neurons_df_all.session == session].recordings_order.iloc[0], seq)
    closedloop_seq = seq[::2]
    openloop_seq = seq[1::2]
    print(closedloop_seq, openloop_seq)
    for i, rec_order in enumerate(closedloop_seq):
        globals()[f"amplitudes_closedloop_{i}"].append(neurons_df_all[select_neurons][neurons_df_all.session == session][f"amplitude_closedloop_recording{rec_order}"].values)
    for i, rec_order in enumerate(openloop_seq):
        globals()[f"amplitudes_openloop_{i}"].append(neurons_df_all[select_neurons][neurons_df_all.session == session][f"amplitude_openloop_recording{rec_order}"].values)

for i in range(5):
    for recording_type in ["closedloop", "openloop"]:
        globals()[f"amplitudes_{recording_type}_{i}"] = [item for sublist in globals()[f"amplitudes_{recording_type}_{i}"] for item in sublist]
        

import scipy.stats as stats
plt.figure(figsize=(10, 5))     
for i in range(2):   
    for itype, recording_type in enumerate(["closedloop", "openloop"]):
        amplitudes = np.array(globals()[f"amplitudes_{recording_type}_{i}"])
        amplitudes[amplitudes>100] = 100
        sns.stripplot(
            x=np.ones(len(amplitudes)) * i * 2 + itype,
            y=amplitudes,
            size=3,
            alpha=0.5,
            jitter=0.2,
            edgecolor="white",
            color=sns.color_palette("Set1")[i * 2 + itype],
        )
        plt.plot(
            [i * 2 + itype - 0.2, i * 2 + itype + 0.2],
            [np.nanmedian(amplitudes), np.nanmedian(amplitudes)],
            linewidth=3,
            color="k"
        )
        plt.yscale("log")
        sns.despine() 
plt.gca().set_xticklabels(["Closed loop", "Open loop"]*2)
        
print(np.median(amplitudes_openloop_0))    
for i in range(2):
    print(np.median(globals()[f"amplitudes_closedloop_{i}"]))
    print(stats.wilcoxon(amplitudes_openloop_0, globals()[f"amplitudes_closedloop_{i}"]))


In [ ]:
amplitudes_all = np.zeros((len(amplitudes_closedloop_0), 3))
amplitudes_all[:,0] = globals()[f"amplitudes_closedloop_0"]
amplitudes_all[:,1] = globals()[f"amplitudes_openloop_0"]
amplitudes_all[:,2] = globals()[f"amplitudes_closedloop_1"]
random_rois = np.random.choice(len(amplitudes_closedloop_0), 100)
plt.plot(amplitudes_all[random_rois].T, 'o-', alpha=0.5, c="k")
plt.yscale("log")

In [ ]:
# import warnings
# from pandas.errors import SettingWithCopyWarning
# warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
# def find_closed_open_pairs(recordings_order, mode="before"):
#     recordings_order = np.array(recordings_order)
#     zero_pos = np.where(recordings_order ==0)[0]
#     one_pos = np.where(recordings_order ==1)[0]
#     zeros, ones = np.meshgrid(zero_pos, one_pos, indexing='ij')
#     # if closed loop is before open loop (ones before zeros)
#     if mode == "before":
#         mask = ones < zeros
#     # if closed loop is after open loop (ones after zeros)
#     elif mode == "after":
#         mask = ones > zeros
#     else:
#         raise ValueError("mode should be 'before' or 'after'")
#     pairs = np.column_stack((ones[mask], zeros[mask]))
#     return pairs

# for order in ["before", "after"]:
#     print(order)
#     globals()[f"pairs_df_{order}"]= pd.DataFrame(columns = [
#         "roi", "session", "iscell", "depth_tuning_test_spearmanr_rval_closedloop", "depth_tuning_test_spearmanr_pval_closedloop", "amplitude_openloop", f"amplitude_closedloop_{order}", "openloop_number", "closedloop_number",
#     ])
#     for session in neurons_df_all.session.unique():
#         neurons_df = neurons_df_all[neurons_df_all.session == session]
#         pairs = find_closed_open_pairs(neurons_df.recordings_order.iloc[0], mode=order)
#         for [closed_idx, open_idx] in pairs:
#             neurons_df["amplitude_openloop"] = neurons_df[f"amplitude_openloop_recording{open_idx}"]
#             neurons_df[f"amplitude_closedloop_{order}"] = neurons_df[f"amplitude_closedloop_recording{closed_idx}"]
#             neurons_df["openloop_number"] = open_idx
#             neurons_df["closedloop_number"] = closed_idx
#             globals()[f"pairs_df_{order}"] = pd.concat([globals()[f"pairs_df_{order}"], neurons_df[["roi", 
#                                         "session", 
#                                         "iscell", 
#                                         "depth_tuning_test_spearmanr_rval_closedloop",
#                                         "depth_tuning_test_spearmanr_pval_closedloop",
#                                         "amplitude_openloop",
#                                         f"amplitude_closedloop_{order}",
#                                         "openloop_number",
#                                         "closedloop_number",
#                                         ]]
#                                 ])
#         print(f"{session} concatenated")

In [ ]:
# amplitude_threshold=100
# for iplot, order in enumerate(["before", "after"]):
#     plt.subplot(2,2,iplot+1)
#     select_neurons = (
#         (globals()[f"pairs_df_{order}"]["iscell"] == 1) &
#         (globals()[f"pairs_df_{order}"]["depth_tuning_test_spearmanr_rval_closedloop"] > 0.1) & 
#         (globals()[f"pairs_df_{order}"]["depth_tuning_test_spearmanr_pval_closedloop"] < 0.05)
#     )
#     globals()[f"pairs_df_{order}"][f"amplitude_closedloop_{order}"][
#         globals()[f"pairs_df_{order}"][f"amplitude_closedloop_{order}"]>amplitude_threshold] = amplitude_threshold
#     globals()[f"pairs_df_{order}"]["amplitude_openloop"][
#         globals()[f"pairs_df_{order}"]["amplitude_openloop"]>amplitude_threshold] = amplitude_threshold
    
#     plt.scatter(globals()[f"pairs_df_{order}"][select_neurons][f"amplitude_closedloop_{order}"], 
#                 globals()[f"pairs_df_{order}"][select_neurons]["amplitude_openloop"],
#                 s=2,
#                 alpha=0.1)
#     plt.xscale("log")
#     plt.yscale("log")
#     plt.gca().set_aspect('equal', adjustable='box')
#     plt.plot([1e-2, 1e2], [1e-2, 1e2], 'k--')
#     plt.xlabel(f"Closed loop amplitude {order}")
#     plt.ylabel("Open loop amplitude")
    
#     plt.subplot(2,2,iplot+3)
#     ratios = globals()[f"pairs_df_{order}"][select_neurons][f"amplitude_closedloop_{order}"]/globals()[f"pairs_df_{order}"][select_neurons]["amplitude_openloop"]
#     bins = np.geomspace(np.nanmin(ratios), np.nanmax(ratios), 30)
#     plt.hist(ratios, 
#              bins=bins, alpha=1)
#     plt.xscale("log")
#     plt.xlabel("Amplitude closedloop / openloop")
#     r, p = stats.wilcoxon(globals()[f"pairs_df_{order}"][select_neurons][f"amplitude_closedloop_{order}"], 
#                           globals()[f"pairs_df_{order}"][select_neurons]["amplitude_openloop"],
#                           alternative="greater")
#     plt.title(f"median {np.round(np.nanmedian(ratios),3)}, pval {p:.3g}", fontsize=10)
#     # plot a vertical line at 1
#     plt.axvline(x=1, color='k', linestyle='--')
#     sns.despine()
    
#     plt.tight_layout()

In [ ]:
def concatenate_decoder_df_for_separate_recordings(flexilims_session, session_list,):
    decoder_df_all = pd.DataFrame(columns=[[
        "session",
        "accuracy_closedloop0",
        "accuracy_openloop1",
        "accuracy_closedloop2",
        "conmat_closedloop0",
        "conmat_openloop1",
        "conmat_closedloop2",
        ]
        ],
        index = range(len(session_list))
    )
    for isess, session in enumerate(session_list):
        print(session)
        neurons_ds = pipeline_utils.create_neurons_ds(
            session_name=session,
            flexilims_session=flexilims_session,
            project=None,
            conflicts="skip",
        )
        neurons_df = pd.read_pickle(neurons_ds.path_full)
        neurons_df = extract_amplitude_separate_recordings(neurons_df)

        if len(neurons_df["recordings_order"].iloc[0]) > 2:
            decoder_df_all.loc[isess, "session"] = session
            for i, rec in enumerate(neurons_df["recordings_order"].iloc[0][:3]):
                if rec == 1:
                    sfx = "closedloop"
                else:
                    sfx = "openloop"
                if os.path.exists(neurons_ds.path_full.parent / f"decoder_results_{sfx}{i}.pickle"):
                    decoder_df = pd.read_pickle(neurons_ds.path_full.parent / f"decoder_results_{sfx}{i}.pickle")
                    if f"accuracy_{sfx}" not in decoder_df.keys():
                        print(f"ERROR: SESSION {session}: {sfx}{i} decoder_results not found")
                    else:
                        decoder_df_all.loc[isess, f"accuracy_{sfx}{i}"] = decoder_df[f"accuracy_{sfx}"]
                        decoder_df_all.loc[isess, f"conmat_{sfx}{i}"] = decoder_df[f"conmat_{sfx}"].reshape(1, decoder_df[f"conmat_{sfx}"].shape[0], decoder_df[f"conmat_{sfx}"].shape[1])
                else:
                    print(f"ERROR: SESSION {session}: {sfx}{i} decoder_results not found")
        else:
            print(f"ERROR: SESSION {session}: not more than 2 recordings")
            
    return decoder_df_all

In [ ]:
decoder_df_all = concatenate_decoder_df_for_separate_recordings(flexilims_session=flexilims_session,
                                                                session_list =  [
                                                                    "PZAH8.2h_S20230224",
                                                                    "PZAH8.2h_S20230303",
                                                                    "PZAH8.2h_S20230314",
                                                                    
                                                                    "PZAH8.2i_S20230203",
                                                                    "PZAH8.2i_S20230209",
                                                                    "PZAH8.2i_S20230216",
                                                                    
                                                                    "PZAH8.2f_S20230214",
                                                                    "PZAH8.2f_S20230313",
 
                                                                    "PZAH10.2f_S20230615", 
                                                                    "PZAH10.2f_S20230822",
                                                                    "PZAH10.2f_S20230908",
                                                                ],)
                                                            

In [ ]:
decoder_df_all

In [ ]:
decoder_df = plt_common_utils.concatenate_all_neurons_df(
    flexilims_session,
    session_list=get_session_list.get_sessions(
        flexilims_session,
        exclude_openloop=False,
        exclude_pure_closedloop=False,
        v1_only=True,
        trialnum_min=10,
        mouse_list=mouse_list,
    ),
    filename="decoder_results.pickle",
    cols=["accuracy_closedloop", "conmat_closedloop", "best_C_closedloop", "acc_speed_bins_closedloop", "conmat_speed_bins_closedloop"],
    read_iscell=False,
    verbose=True,
)

decoder_df.to_pickle(SAVE_ROOT / "decoder_df_all.pickle")

In [ ]:
project = "hey2_3d-vision_foodres_20220101"
session_name = "PZAH10.2f_S20230822"
flexilims_session = flz.get_flexilims_session(project)

neurons_ds = pipeline_utils.create_neurons_ds(
    session_name=session_name,
    flexilims_session=flexilims_session,
    project=None,
    conflicts="skip",
)

decoder_df = pd.read_pickle(neurons_ds.path_full.parent / "decoder_results_closedloop0.pickle")
neurons_df = pd.read_pickle(neurons_ds.path_full)

In [ ]:
neurons_df.recordings_order

In [ ]:
import os
os.listdir(neurons_ds.path_full.parent)